# YOLO Vehicle Detection Training on Google Colab

## 🚀 Features
- ใช้ GPU ฟรีจาก Google Colab
- Mount Google Drive เพื่อบันทึกผลลัพธ์
- ดาวน์โหลด Dataset จาก Roboflow
- Training YOLOv11 สำหรับ Vehicle Detection
- บันทึก Model และผลลัพธ์ลง Google Drive อัตโนมัติ

## 1️⃣ Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# สร้างโฟลเดอร์สำหรับเก็บผลลัพธ์
DRIVE_PATH = '/content/drive/MyDrive/YOLO_Training'
os.makedirs(DRIVE_PATH, exist_ok=True)

print(f"✅ Google Drive mounted successfully!")
print(f"📁 Results will be saved to: {DRIVE_PATH}")

## 2️⃣ ตรวจสอบ GPU

In [ ]:
import torch

print("🔍 GPU Information:")
print(f"   PyTorch Version: {torch.__version__}")
print(f"   CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("   ⚠️ No GPU detected! Training will be slow.")

## 3️⃣ ติดตั้ง Dependencies

In [ ]:
!pip install ultralytics roboflow -q

print("✅ Dependencies installed!")

## 4️⃣ ดาวน์โหลด Dataset จาก Roboflow

In [ ]:
from roboflow import Roboflow

# Roboflow Configuration
ROBOFLOW_API_KEY = "xWmGZqldGGdVHAeTBz8b"
WORKSPACE_NAME = "thaidetec"
PROJECT_NAME = "vehicle-detection-yg4le"
VERSION_NUMBER = 12
DATASET_FORMAT = "yolov11"

print("📥 Downloading dataset from Roboflow...")
print(f"   Workspace: {WORKSPACE_NAME}")
print(f"   Project: {PROJECT_NAME}")
print(f"   Version: {VERSION_NUMBER}")
print()

# Initialize Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(WORKSPACE_NAME).project(PROJECT_NAME)
version = project.version(VERSION_NUMBER)

# Download dataset
dataset = version.download(DATASET_FORMAT)

# Get dataset path
dataset_path = dataset.location
data_yaml = f"{dataset_path}/data.yaml"

print(f"\n✅ Dataset downloaded successfully!")
print(f"   Location: {dataset_path}")
print(f"   Config: {data_yaml}")

## 5️⃣ Training Configuration

In [ ]:
# =============================================================================
# TRAINING CONFIGURATION - ปรับแต่งค่าที่นี่
# =============================================================================

# Model Configuration
MODEL_NAME = "yolo11l.pt"  # Options: yolo11n.pt, yolo11s.pt, yolo11m.pt, yolo11l.pt, yolo11x.pt

# Training Hyperparameters
EPOCHS = 100              # จำนวน Epochs
BATCH_SIZE = 16           # Batch size (ลดลงถ้า GPU memory ไม่พอ)
IMAGE_SIZE = 640          # ขนาดภาพ (640 หรือ 1280)
PATIENCE = 50             # จำนวน epochs ที่จะหยุดถ้าไม่มีการปรับปรุง

# Device Configuration
DEVICE = 0                # GPU ID (0 = GPU แรก)

# Output Configuration
PROJECT_NAME = "runs/detect"
RUN_NAME = "train"        # ชื่อ run

# Advanced Settings
WORKERS = 2               # จำนวน workers (Colab แนะนำ 2)
CACHE = True              # Cache images
AMP = True                # Automatic Mixed Precision

print("📋 Training Configuration:")
print("=" * 60)
print(f"   Model:      {MODEL_NAME}")
print(f"   Epochs:     {EPOCHS}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Image Size: {IMAGE_SIZE}")
print(f"   Device:     GPU {DEVICE}")
print("=" * 60)

## 6️⃣ Auto Resume / Fine-tune Training

> **วิธีใช้:**
> - `RESUME_IF_FOUND = True` + `EXTRA_EPOCHS = 0` → Resume ต่อจากที่หลุดกลางคัน
> - `EXTRA_EPOCHS = 50` → Fine-tune เพิ่มอีก 50 epochs (เช่น จาก 50 เป็น 100)

In [ ]:
from ultralytics import YOLO
import glob, os, re

# -------------------------
# A) หา checkpoint ล่าสุดใน Drive
# -------------------------
def find_latest_last_pt(drive_path):
    cks = sorted(glob.glob(f"{drive_path}/train_*/weights/last.pt"), reverse=True)
    return cks[0] if cks else None

def parse_train_folder_from_ckpt(ckpt_path):
    # .../YOLO_Training/train_YYYYMMDD_HHMMSS/weights/last.pt
    return os.path.dirname(os.path.dirname(ckpt_path))

ckpt = find_latest_last_pt(DRIVE_PATH)

# -------------------------
# B) เลือกโหมด
# -------------------------
# 1) ถ้าหลุดกลางคัน -> resume ต่อเลย (ต่อ epoch เดิม)
RESUME_IF_FOUND = True

# 2) ถ้าอยาก "เพิ่มจาก 50 เป็น 100" ให้ใช้ fine-tune เพิ่มอีก 50
# ตั้งค่าเพิ่ม epoch ตรงนี้ (0 = ไม่เพิ่ม)
EXTRA_EPOCHS = 0   # เช่น เทรนมา 50 แล้วอยากรวมเป็น 100 => ใส่ 50

print("=" * 60)
print("🔍 ตรวจสอบ Checkpoint ใน Google Drive...")
print(f"   Checkpoint: {ckpt}")
print("=" * 60)

if ckpt and RESUME_IF_FOUND and EXTRA_EPOCHS == 0:
    print("\n🔄 Resume ต่อจากที่ค้างไว้ (หลุดกลางคัน)")
    model = YOLO(ckpt)
    results = model.train(resume=True)

elif ckpt and EXTRA_EPOCHS > 0:
    print(f"\n➕ Fine-tune เพิ่มอีก {EXTRA_EPOCHS} epochs (ใช้ต่อยอดจาก last.pt)")
    model = YOLO(ckpt)
    results = model.train(
        data=data_yaml,
        epochs=EXTRA_EPOCHS,     # เทรนเพิ่มอีกกี่ epoch
        batch=BATCH_SIZE,
        imgsz=IMAGE_SIZE,
        device=DEVICE,
        project=PROJECT_NAME,
        name=f"{RUN_NAME}_plus{EXTRA_EPOCHS}",
        patience=PATIENCE,
        workers=WORKERS,
        cache=CACHE,
        amp=AMP,
        save=True,
        save_period=10,
        plots=True,
        verbose=True,
    )

else:
    print("\n🆕 ไม่พบ checkpoint -> เริ่มเทรนใหม่")
    model = YOLO(MODEL_NAME)
    results = model.train(
        data=data_yaml,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMAGE_SIZE,
        device=DEVICE,
        project=PROJECT_NAME,
        name=RUN_NAME,
        patience=PATIENCE,
        workers=WORKERS,
        cache=CACHE,
        amp=AMP,
        save=True,
        save_period=10,
        plots=True,
        verbose=True,
    )

latest_run = str(results.save_dir)
print("\n" + "=" * 60)
print("✅ Training เสร็จสิ้น!")
print(f"   latest_run = {latest_run}")
print("=" * 60)

## 7️⃣ บันทึกผลลัพธ์ลง Google Drive

In [ ]:
import shutil
from datetime import datetime
import os

# สร้างชื่อโฟลเดอร์ตามวันเวลา
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_folder = f"{DRIVE_PATH}/train_{timestamp}"

# หา path ของ run ล่าสุด
latest_run = results.save_dir

print(f"💾 Saving results to Google Drive...")
print(f"   Source: {latest_run}")
print(f"   Destination: {save_folder}")
print()

# Copy ทั้งโฟลเดอร์ไปยัง Google Drive
shutil.copytree(latest_run, save_folder)

print("✅ Results saved successfully!")
print()
print("📁 Saved files:")
print(f"   📊 Best Model: {save_folder}/weights/best.pt")
print(f"   📊 Last Model: {save_folder}/weights/last.pt")
print(f"   📈 Results: {save_folder}/results.png")
print(f"   📈 Confusion Matrix: {save_folder}/confusion_matrix.png")
print()
print(f"🎉 All done! Check your Google Drive at: {save_folder}")

## 8️⃣ แสดงผลลัพธ์

In [ ]:
from IPython.display import Image, display
import os

# แสดงกราฟผลลัพธ์
results_img = f"{latest_run}/results.png"
confusion_matrix_img = f"{latest_run}/confusion_matrix.png"

if os.path.exists(results_img):
    print("📊 Training Results:")
    display(Image(filename=results_img))

if os.path.exists(confusion_matrix_img):
    print("\n📊 Confusion Matrix:")
    display(Image(filename=confusion_matrix_img))

## 9️⃣ (Optional) Validate Model

In [ ]:
# Validate best model
best_model_path = f"{latest_run}/weights/best.pt"
model = YOLO(best_model_path)

print("🔍 Validating model...")
metrics = model.val(data=data_yaml)

print("\n📊 Validation Metrics:")
print(f"   mAP50:     {metrics.box.map50:.4f}")
print(f"   mAP50-95:  {metrics.box.map:.4f}")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall:    {metrics.box.mr:.4f}")

## 📝 Notes

### Tips:
- ถ้า GPU memory ไม่พอ ให้ลด `BATCH_SIZE`
- ถ้าต้องการ Model ที่เล็กกว่า ให้เปลี่ยน `MODEL_NAME` เป็น `yolo11n.pt` หรือ `yolo11s.pt`
- ถ้าต้องการ Model ที่ใหญ่กว่า ให้เปลี่ยนเป็น `yolo11x.pt`
- ผลลัพธ์จะถูกบันทึกอัตโนมัติลง Google Drive

### Model Sizes:
- `yolo11n.pt` - Nano (เล็กที่สุด, เร็วที่สุด)
- `yolo11s.pt` - Small
- `yolo11m.pt` - Medium
- `yolo11l.pt` - Large (ค่าเริ่มต้น)
- `yolo11x.pt` - Extra Large (ใหญ่ที่สุด, แม่นที่สุด)